In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

參考

https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing/notebook

# 讀取資料

In [ ]:
##0519前處理
df = pd.read_csv("/content/drive/MyDrive/佩玲論文資料/alldata_clean.csv",encoding = 'unicode_escape')
df 

,company,sentence
0,Amedisys,The risks described below and risks described ...
1,Amedisys,The risk factors described below and elsewhere...
2,Amedisys,Our business and consolidated financial condit...
3,Amedisys,If any of the following risks are actually rea...
4,Amedisys,In that case the trading price of our common s...
...,...,...
4785,UnitedHealth,We are also required by law or regulation to m...
4786,UnitedHealth,The levels of capitalization required depend p...
4787,UnitedHealth,In most states we are required to seek approva...
4788,UnitedHealth,An inability of our regulated subsidiaries to ...


In [ ]:
#跟之前一樣，將 <20 字元的資料筆數刪除
fuckdf_idx=df[df["sentence"].str.len() < 20].index
fuckdrop=df.drop(fuckdf_idx)
fuckdrop=df.drop(fuckdf_idx)
df = fuckdrop
fuckdrop.reset_index(inplace=True, drop=True)
df = fuckdrop


In [ ]:
#轉換大小寫
df["sentence"] = df["sentence"].str.lower()
df

,company,sentence
0,Amedisys,the risks described below and risks described ...
1,Amedisys,the risk factors described below and elsewhere...
2,Amedisys,our business and consolidated financial condit...
3,Amedisys,if any of the following risks are actually rea...
4,Amedisys,in that case the trading price of our common s...
...,...,...
4735,UnitedHealth,we are also required by law or regulation to m...
4736,UnitedHealth,the levels of capitalization required depend p...
4737,UnitedHealth,in most states we are required to seek approva...
4738,UnitedHealth,an inability of our regulated subsidiaries to ...


In [ ]:
#清理資料(去除標點符號)
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["sentence"].apply(lambda text: remove_punctuation(text))
df.head()

,company,sentence,text_wo_punct
0,Amedisys,the risks described below and risks described ...,the risks described below and risks described ...
1,Amedisys,the risk factors described below and elsewhere...,the risk factors described below and elsewhere...
2,Amedisys,our business and consolidated financial condit...,our business and consolidated financial condit...
3,Amedisys,if any of the following risks are actually rea...,if any of the following risks are actually rea...
4,Amedisys,in that case the trading price of our common s...,in that case the trading price of our common s...


#前處理
刪除停用字

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#查看nltk內的停用字
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

,company,sentence,text_wo_punct,text_wo_stop
0,Amedisys,the risks described below and risks described ...,the risks described below and risks described ...,risks described risks described elsewhere form...
1,Amedisys,the risk factors described below and elsewhere...,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...
2,Amedisys,our business and consolidated financial condit...,our business and consolidated financial condit...,business consolidated financial condition resu...
3,Amedisys,if any of the following risks are actually rea...,if any of the following risks are actually rea...,following risks actually realized business con...
4,Amedisys,in that case the trading price of our common s...,in that case the trading price of our common s...,case trading price common stock could decline


去除頻繁字

In [ ]:
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

In [ ]:
cnt.most_common(2)


[('may', 1551), ('could', 1288)]

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(2)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

,company,sentence,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,Amedisys,the risks described below and risks described ...,the risks described below and risks described ...,risks described risks described elsewhere form...,risks described risks described elsewhere form...
1,Amedisys,the risk factors described below and elsewhere...,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...,risk factors described elsewhere form 10k risk...
2,Amedisys,our business and consolidated financial condit...,our business and consolidated financial condit...,business consolidated financial condition resu...,business consolidated financial condition resu...
3,Amedisys,if any of the following risks are actually rea...,if any of the following risks are actually rea...,following risks actually realized business con...,following risks actually realized business con...
4,Amedisys,in that case the trading price of our common s...,in that case the trading price of our common s...,case trading price common stock could decline,case trading price common stock decline


In [ ]:
#出現最少次
#cnt.most_common()[:-20-1:-1]

[('choosing', 1),
 ('nces', 1),
 ('conseque', 1),
 ('statistically', 1),
 ('administers', 1),
 ('multifaceted', 1),
 ('chile', 1),
 ('dol', 1),
 ('clinicbased', 1),
 ('agreeing', 1),
 ('le', 1),
 ('wholesa', 1),
 ('forecasted', 1),
 ('ison', 1),
 ('compar', 1),
 ('ould', 1),
 ('match', 1),
 ('bonuses', 1),
 ('stars', 1),
 ('incorrect', 1)]

In [ ]:
# df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

# n_rare_words = 10
# RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
# def remove_rarewords(text):
#    """custom function to remove the rare words"""
#    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

# df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
# df.drop(["text_wo_stopfreqrare"], axis=1, inplace=True)
#df.head()


,company,sentence,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,Amedisys,the risks described below and risks described ...,the risks described below and risks described ...,risks described risks described elsewhere form...,risks described risks described elsewhere form...
1,Amedisys,the risk factors described below and elsewhere...,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...,risk factors described elsewhere form 10k risk...
2,Amedisys,our business and consolidated financial condit...,our business and consolidated financial condit...,business consolidated financial condition resu...,business consolidated financial condition resu...
3,Amedisys,if any of the following risks are actually rea...,if any of the following risks are actually rea...,following risks actually realized business con...,following risks actually realized business con...
4,Amedisys,in that case the trading price of our common s...,in that case the trading price of our common s...,case trading price common stock could decline,case trading price common stock decline


#Stemming 詞幹提取

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# # Drop the two columns 

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])

# df["text_stemmed"] = df["text_wo_stopfreq"].apply(lambda text: stem_words(text))
# df.head()

In [ ]:
stemmer2 = SnowballStemmer("english")
def stem_words2(text):
    return " ".join([stemmer2.stem(word) for word in text.split()])
df["text_SnowballStemmer"] = df["text_wo_stop"].apply(lambda text: stem_words2(text))
df.head()

,company,sentence,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_SnowballStemmer
0,Amedisys,the risks described below and risks described ...,the risks described below and risks described ...,risks described risks described elsewhere form...,risks described risks described elsewhere form...,risk describ risk describ elsewher form 10k co...
1,Amedisys,the risk factors described below and elsewhere...,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...,risk factors described elsewhere form 10k risk...,risk factor describ elsewher form 10k risk fac...
2,Amedisys,our business and consolidated financial condit...,our business and consolidated financial condit...,business consolidated financial condition resu...,business consolidated financial condition resu...,busi consolid financi condit result oper cash ...
3,Amedisys,if any of the following risks are actually rea...,if any of the following risks are actually rea...,following risks actually realized business con...,following risks actually realized business con...,follow risk actual realiz busi consolid financ...
4,Amedisys,in that case the trading price of our common s...,in that case the trading price of our common s...,case trading price common stock could decline,case trading price common stock decline,case trade price common stock could declin


In [ ]:
df.drop(["text_wo_stop","text_wo_punct"], axis=1, inplace=True)

In [ ]:
df.head()

,company,sentence,text_wo_stopfreq,text_SnowballStemmer
0,Amedisys,the risks described below and risks described ...,risks described risks described elsewhere form...,risk describ risk describ elsewher form 10k co...
1,Amedisys,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...,risk factor describ elsewher form 10k risk fac...
2,Amedisys,our business and consolidated financial condit...,business consolidated financial condition resu...,busi consolid financi condit result oper cash ...
3,Amedisys,if any of the following risks are actually rea...,following risks actually realized business con...,follow risk actual realiz busi consolid financ...
4,Amedisys,in that case the trading price of our common s...,case trading price common stock decline,case trade price common stock could declin


#Lemmatization

詞幹還原


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text_SnowballStemmer"].apply(lambda text: lemmatize_words(text))
df.head()

,company,sentence,text_wo_stopfreq,text_SnowballStemmer,text_lemmatized
0,Amedisys,the risks described below and risks described ...,risks described risks described elsewhere form...,risk describ risk describ elsewher form 10k co...,risk describ risk describ elsewher form 10k co...
1,Amedisys,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...,risk factor describ elsewher form 10k risk fac...,risk factor describ elsewher form 10k risk fac...
2,Amedisys,our business and consolidated financial condit...,business consolidated financial condition resu...,busi consolid financi condit result oper cash ...,busi consolid financi condit result oper cash ...
3,Amedisys,if any of the following risks are actually rea...,following risks actually realized business con...,follow risk actual realiz busi consolid financ...,follow risk actual realiz busi consolid financ...
4,Amedisys,in that case the trading price of our common s...,case trading price common stock decline,case trade price common stock could declin,case trade price common stock could declin


In [ ]:
df.to_csv('/content/drive/MyDrive/佩玲論文資料/data_詞幹還原.csv', encoding = 'utf-8',index = False)